## Imports

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import time 

In [2]:
chart = pd.read_csv('../regional-global-daily-latest.csv', header = 1)
chart.head()

,Position,Track Name,Artist,Streams,URL
0,1,DÁKITI,Bad Bunny,6016657,https://open.spotify.com/track/4MzXwWMhyBbmu6h...
1,2,All I Want for Christmas Is You,Mariah Carey,5902919,https://open.spotify.com/track/0bYg9bo50gSsH3L...
2,3,Last Christmas,Wham!,5153859,https://open.spotify.com/track/2FRnf9qhLbvw8fu...
3,4,Santa Tell Me,Ariana Grande,4202883,https://open.spotify.com/track/0lizgQ7Qw35od7C...
4,5,It's Beginning to Look a Lot like Christmas,Michael Bublé,4074711,https://open.spotify.com/track/0lLdorYw7lVrJyd...


In [3]:
song_uris = []
for url in chart['URL']:
     song_uris.append(url[31:])
chart['song_uri'] = song_uris

In [4]:
chart.head()

,Position,Track Name,Artist,Streams,URL,song_uri
0,1,DÁKITI,Bad Bunny,6016657,https://open.spotify.com/track/4MzXwWMhyBbmu6h...,4MzXwWMhyBbmu6hOcLVD49
1,2,All I Want for Christmas Is You,Mariah Carey,5902919,https://open.spotify.com/track/0bYg9bo50gSsH3L...,0bYg9bo50gSsH3LtXe2SQn
2,3,Last Christmas,Wham!,5153859,https://open.spotify.com/track/2FRnf9qhLbvw8fu...,2FRnf9qhLbvw8fu4IBXx78
3,4,Santa Tell Me,Ariana Grande,4202883,https://open.spotify.com/track/0lizgQ7Qw35od7C...,0lizgQ7Qw35od7CYaoMBZb
4,5,It's Beginning to Look a Lot like Christmas,Michael Bublé,4074711,https://open.spotify.com/track/0lLdorYw7lVrJyd...,0lLdorYw7lVrJydTINhWdI


## Credentials 

In [5]:
creds= pd.read_csv('../credentials')
SPOTIPY_CLIENT_ID = creds['Client ID'][0]
SPOTIPY_CLIENT_SECRET = creds['Client Secret'][0]
#Shameless redirect to my dance group if you're signing in for the first time
SPOTIPY_REDIRECT_URI= 'https://www.ballethermosoamanecer.com/'
username = ""

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id= SPOTIPY_CLIENT_ID, 
                                                      client_secret=SPOTIPY_CLIENT_SECRET)

sp = spotipy.Spotify(client_credentials_manager= client_credentials_manager)
scope = 'playlist-read-collaborative user-library-read playlist-read-private playlist-modify-public playlist-modify-private user-read-recently-played user-top-read'
token = util.prompt_for_user_token(username, scope, 
                                   client_id= SPOTIPY_CLIENT_ID, 
                                   client_secret= SPOTIPY_CLIENT_SECRET, 
                                   redirect_uri= SPOTIPY_REDIRECT_URI)
if token:
    sp = spotipy.Spotify(auth=token)
    track_audio_features = []
    artist_uri = []
    artist_name = []
    artist_set = set()
    alb_date = []
    alb_name = [] 
    art_uri = []
    alb_genre = []
    loops = 0
    a_count = 0 
    
    print(f'There are {len(song_uris)} songs that we found')
    while 20* (loops+1) <= len(song_uris):
        #get audio features in batches of 20
        track_audio_features.extend(sp.audio_features(song_uris[20*loops :20*(loops+1)]))
        
        #getting artist information 
        temp= [i for i in sp.tracks(song_uris[20*loops :20*(loops + 1)])['tracks'] ]
        for index in range(len(temp)):
            to_add = [artist['id'] for artist in temp[index]['artists']]
            artist_uri.append(to_add)
            artist_name.append([artist['name'] for artist in temp[index]['artists']])
            #if there was a collab, collect artist information
            if len(to_add) > 1:
                for artist in temp[index]['artists']:
                    artist_set.add(artist['id'])
        loops += 1
        print(f'Found audio features for {20*(loops+1)} songs')
        
    print('Creating Dataframe')
    audio_features = pd.DataFrame(track_audio_features)
    chart['artist_uri'] = artist_uri
    
    print("Merging Dataframe")
    df = chart.merge(audio_features, left_on = 'song_uri', right_on= 'id')
    
    print(f'There are {len(artist_set)} artists that we found')
    for uri in artist_set: 
        for i in range(len(sp.artist_albums(uri)['items'])):
            art_uri.append(uri)
            alb_date.append(sp.artist_albums(uri)['items'][i]['release_date'])
            alb_name.append(sp.artist_albums(uri)['items'][i]['name'])             
#             alb_genre.append(sp.artist_albums(uri)['items'][i]['genres'])
        a_count += 1 
        if a_count % 10 == 0:     
            print(f'Found album features for {a_count} artists')
    
    print("Creating Artist Dataframe")
    album_data = pd.DataFrame({'Artist URI': art_uri,
                           'Album Name': alb_name,
                           'Album Release Date' : alb_date
#                                'Album Generes': alb_genres
                          })
    
else:
    print("Can't get token for", username)
    
    
    

There are 200 songs that we found
Found audio features for 40 songs
Found audio features for 60 songs
Found audio features for 80 songs
Found audio features for 100 songs
Found audio features for 120 songs
Found audio features for 140 songs
Found audio features for 160 songs
Found audio features for 180 songs
Found audio features for 200 songs
Found audio features for 220 songs
Creating Dataframe
Merging Dataframe
There are 129 artists that we found
Found album features for 10 artists
Found album features for 20 artists
Found album features for 30 artists
Found album features for 40 artists
Found album features for 50 artists


In [ ]:
# df['collaborators'] = df['artist_uri'].apply(lambda x : len(x))

In [ ]:
df[df['collaborators']>1]

In [ ]:
# df.drop(['URL', 'song_uri', 'type', 'uri', 'track_href'])

In [ ]:
df.columns

In [ ]:
album_data